In [1]:
import bigframes.pandas as bpd

# Set to your BigQuery GCP project.
bpd.options.bigquery.project = "bigframes-dev"

In [2]:
df = bpd.read_json(
    "gs://cloud-samples-data/third-party/usa-loc-national-jukebox/jukebox.jsonl",
    engine="bigquery",
    orient="records",
    lines=True,
)
df.shape

/home/swast/src/github.com/tswast/code-snippets/venv/lib/python3.12/site-packages/bigframes/core/global_session.py:103: DefaultLocationWarning: No explicit location is set, so using location US for the session.
  _global_session = bigframes.session.connect(


(100, 19)

In [3]:
df.peek()

,URL,Recording Repository,Recording Label,Recording Take Number,Recording Date,Part of,Names,Recording Matrix Number,Recording Catalog Number,Media Size,MP3 URL,IIIF Presentation Manifest,Genre,Other Title,Recording Location,Online Format,Summary,Rights Advisory,Title
69,https://www.loc.gov/item/jukebox-9538/,['Source of original recording: Department of ...,['Victor'],['1'],['1925-04-30'],"['Department of Special Collections, Davidson ...","['Collazo, Ramón -- Composer -- Composer' 'Ca...",['BA-591 (Matrix ID)'],['79553'],['10-in.'],https://tile.loc.gov/streaming-services/iiif/s...,['Manifest (JSON/LD)'],['Spanish (Argentina)' 'Ethnic music' 'Spanish...,['Tango'],"['Buenos Aires, Argentina [unconfirmed]']",['audio' 'image'],['Instrumental ensemble'],['Inclusion of the recording in the National J...,['Volveras']
50,https://www.loc.gov/item/jukebox-767050/,['Source of original recording: Department of ...,['Columbia'],['2'],['1925-12-31'],"['Department of Special Collections, Davidson ...","['Foster, Stephen Collins -- Composer -- Compo...",['90141 (Matrix ID)'],['A3109'],['10-in.'],https://tile.loc.gov/streaming-services/iiif/s...,['Manifest (JSON/LD)'],[],['My old Kentucky home'],[],['audio' 'image'],['Band'],['Inclusion of the recording in the National J...,['Battle hymn of the Republic']
72,https://www.loc.gov/item/jukebox-744502/,['Source of original recording: Department of ...,['Columbia'],['2'],['1925-03-30'],"['Department of Special Collections, Davidson ...","['Leal, Gustavo -- Vocalist -- Tenor Vocal']",['105566 (Matrix ID)'],['1007-X'],['10-in.'],https://tile.loc.gov/streaming-services/iiif/s...,['Manifest (JSON/LD)'],['Portuguese'],['Cancao popular'],[],['audio' 'image'],"['Male vocal solo, with orchestra']",['Inclusion of the recording in the National J...,['Fado do padeiro']
89,https://www.loc.gov/item/jukebox-9648/,['Source of original recording: Department of ...,['Victor'],['2'],['1925-11-09'],"['Department of Special Collections, Davidson ...",['Orquesta Típica Victor -- Musical Group -- M...,['BA-703 (Matrix ID)'],['79608'],['10-in.'],https://tile.loc.gov/streaming-services/iiif/s...,['Manifest (JSON/LD)'],['Spanish (Argentina)' 'Ethnic music' 'Spanish...,['Tango'],"['Buenos Aires, Argentina [unconfirmed]']",['audio' 'image'],['Instrumental ensemble'],['Inclusion of the recording in the National J...,['Sarandi']
48,https://www.loc.gov/item/jukebox-767143/,['Source of original recording: Department of ...,['Columbia'],['2'],['1925-12-31'],"['Department of Special Collections, Davidson ...",['Columbia Band -- Musical Group -- Musical Gr...,['90134 (Matrix ID)'],['A3114'],['10-in.'],https://tile.loc.gov/streaming-services/iiif/s...,['Manifest (JSON/LD)'],['Educational'],"['Come, Thou almighty King']",[],['audio' 'image'],['Band'],['Inclusion of the recording in the National J...,"[""Love's old sweet song""]"


In [4]:
# Flatten lists where we (probably?) only have one item.
flattened = df.explode([
    "Recording Repository",
    "Recording Label",
    "Recording Take Number",
    "Recording Date",
    "Recording Matrix Number",
    "Recording Catalog Number",
    "Media Size",
    "Recording Location",
    "Summary",
    "Rights Advisory",
    "Title",
])
flattened.shape

(100, 19)

In [6]:
flattened.peek()

,URL,Recording Repository,Recording Label,Recording Take Number,Recording Date,Part of,Names,Recording Matrix Number,Recording Catalog Number,Media Size,MP3 URL,IIIF Presentation Manifest,Genre,Other Title,Recording Location,Online Format,Summary,Rights Advisory,Title
20,https://www.loc.gov/item/jukebox-73006/,Source of original recording: Department of Sp...,Victor,2,1925-03-02,"['Department of Special Collections, Davidson ...","['Medoff, David -- Vocalist -- Tenor Vocal' '...",B-32034 (Matrix ID),77981,10-in.,https://tile.loc.gov/streaming-services/iiif/s...,['Manifest (JSON/LD)'],['Russian' 'Ethnic music' 'Russian'],"['Zaitshek' ""The rabbit's footsteps"" 'Chiberia...","New York, New York",['audio' 'image'],"Male vocal solo, with orchestra",Inclusion of the recording in the National Juk...,Zaichik
4,https://www.loc.gov/item/jukebox-675580/,Source of original recording: Department of Sp...,Columbia,1,1925-02-27,"['Department of Special Collections, Davidson ...","['Cavaliers, The -- Musical Group -- Musical G...",140403 (Matrix ID),331-D,10-in.,https://tile.loc.gov/streaming-services/iiif/s...,['Manifest (JSON/LD)'],[],[],"New York, New York",['audio' 'image'],Jazz/dance band,Inclusion of the recording in the National Juk...,The midnight waltz
6,https://www.loc.gov/item/jukebox-9599/,Source of original recording: Department of Sp...,Victor,1,1925-09-02,"['Department of Special Collections, Davidson ...","['Piana, Sebastián -- Composer -- Composer' '...",BA-652 (Matrix ID),79584,10-in.,https://tile.loc.gov/streaming-services/iiif/s...,['Manifest (JSON/LD)'],['Spanish (Argentina)' 'Ethnic music' 'Spanish...,[],"Buenos Aires, Argentina [unconfirmed]",['audio' 'image'],Instrumental ensemble,Inclusion of the recording in the National Juk...,Sobre el pucho
12,https://www.loc.gov/item/jukebox-675604/,Source of original recording: Department of Sp...,Columbia,5,1925-03-21,"['Department of Special Collections, Davidson ...","['Mandoliers, The -- Musical Group -- Musical ...",W140409 (Matrix ID),345-D,10-in.,https://tile.loc.gov/streaming-services/iiif/s...,['Manifest (JSON/LD)'],[],['Waltz'],"New York, New York",['audio' 'image'],"Jazz/dance band, with male vocal solo",Inclusion of the recording in the National Juk...,Only a weaver of dreams
63,https://www.loc.gov/item/jukebox-9670/,Source of original recording: Department of Sp...,Victor,1,1925-10-31,"['Department of Special Collections, Davidson ...","['Laina, José -- Composer -- Composer' 'Orque...",BA-725 (Matrix ID),79618,10-in.,https://tile.loc.gov/streaming-services/iiif/s...,['Manifest (JSON/LD)'],['Spanish (Argentina)' 'Ethnic music' 'Spanish...,['Tango'],"Buenos Aires, Argentina [unconfirmed]",['audio' 'image'],Instrumental ensemble,Inclusion of the recording in the National Juk...,Y te deje llorando


In [7]:
flattened = flattened.assign(**{
    "GCS Prefix": "gs://cloud-samples-data/third-party/usa-loc-national-jukebox/",
    "GCS Stub": flattened['URL'].str.extract(r'/(jukebox-[0-9]+)/'),
})
flattened.cache()
flattened["GCS URI"] = flattened["GCS Prefix"] + flattened["GCS Stub"] + ".mp3"
flattened["GCS Blob"] = flattened["GCS URI"].str.to_blob()

In [8]:
flattened["Transcription"] = flattened["GCS Blob"].blob.audio_transcribe(
    model_name="gemini-2.0-flash-001",
    verbose=True,
)
flattened["Transcription"]

/home/swast/src/github.com/tswast/code-snippets/venv/lib/python3.12/site-packages/bigframes/core/array_value.py:108: PreviewWarning: JSON column interpretation as a custom PyArrow extention in
`db_dtypes` is a preview feature and subject to change.
  warnings.warn(msg, bfe.PreviewWarning)


0     {'status': '', 'content': "Somebody's trying t...
1                        {'status': '', 'content': '.'}
2     {'status': '', 'content': "Softly the pale blo...
3     {'status': '', 'content': "It's goodbye you sa...
4                        {'status': '', 'content': '.'}
5                        {'status': '', 'content': '.'}
6                         {'status': '', 'content': ''}
7     {'status': '', 'content': "Oh look at father t...
8     {'status': '', 'content': 'Una indita y un ran...
9                         {'status': '', 'content': ''}
10                       {'status': '', 'content': '.'}
11    {'status': '', 'content': 'Где ты, блаженство ...
12    {'status': '', 'content': "Only a weaver of dr...
13                     {'status': '', 'content': '.\n'}
14                    {'status': '', 'content': 'Woo!'}
15                     {'status': '', 'content': '.\n'}
16    {'status': '', 'content': 'Prišiel ten opar, p...
17                       {'status': '', 'content

In [10]:
flattened["Transcription"].struct.field("content")

0     Somebody's trying to take you away, trying to ...
1                                                     .
2     Softly the pale blossoms fall, here by the old...
3     It's goodbye you said, parting of the way.\nMe...
4                                                     .
5                                                     .
6                                                      
7     Oh look at father throwing us on.\nGoodbye mou...
8     Una indita y un ranchero\nfueron a la zanja a ...
9                                                      
10                                                    .
11    Где ты, блаженство прежних дней?\nУвы, давно в...
12    Only a weaver of dreams, all and dreams, dream...
13                                                  .\n
14                                                 Woo!
15                                                  .\n
16    Prišiel ten opar, príroda usnulá zas,\nna okie...
17                                              

In [11]:
(flattened["Transcription"].struct.field("status") == "").sum()

np.int64(100)

In [13]:
flattened.shape

(100, 24)